In [31]:
import os, sys, json
from pathlib import Path

from tqdm import tqdm
import pandas as pd
import pandas.io.sql as sqlio

# datascience tools to connect to dbs
sys.path.append("/Users/hussein.alolimat/work/codes/tempus-machine-learning/")
from tools.tempuslabs.db_connector import db_connector as dbc

# Read the database connection info from your secrets file:
with open("/Users/hussein.alolimat/.ssh/tempus_secrets.json", 'rb') as file:
    secrets = json.load(file)

In [32]:
# connect to clq
db_clq = dbc.AutoConnector('clq', secrets['db']['clq'])
conn_clq = db_clq.auto_connect()

Using JSON object to set connection parameters.


In [33]:
df=pd.read_csv("/Users/hussein.alolimat/Downloads/CLQ_Cases_Possible_NGS_Reports.csv")

In [8]:
results=[]
for i, row in tqdm(df.iterrows()):
    qry = f"""
    SELECT partner_filename, mime_type
    FROM vault.clinical_document_gateway_meta_sat AS a
           JOIN vault.patient_clinical_document_link AS b
             ON a.clinical_document_hkey = b.clinical_document_hkey
        where b.patient_tempus_id='{row.patient_id}';
    """
    fileNames=set()
    for _, r in sqlio.read_sql_query(qry, conn_clq).iterrows():
        fileNames.add(r.partner_filename.split(".")[-1])
        fileNames.add(r.mime_type)
    
    results.append([row.patient_id, " -".join(fileNames)])

830it [03:09,  4.38it/s]


In [9]:
results

[['3aba39e9-934d-44b9-92d2-4b1d67749272', 'application/rtf -rtf'],
 ['7ecb01df-4cdb-4f3a-a6a1-a6403d04ba31', ' -Rich Text -txt'],
 ['11520710-9145-4f67-8a79-e7ebf0c2ebba', ' -txt'],
 ['d48ff340-65e7-4de7-b9f2-439a20f11502', ' -rtf'],
 ['d88f8274-e9a2-4a6f-a190-27df6982a88a', ' -pdf -application/pdf -txt'],
 ['fc33de31-e8d9-46d6-b987-b49986109af7', ' -txt'],
 ['ceeac743-ed06-492c-a43b-87dad97be0e1', ' -txt'],
 ['a3cec039-87aa-4435-a6ea-07128bd6e5ba', 'application/rtf -rtf'],
 ['7432ca13-8e28-4a0b-a159-640213b4b327',
  'Rich Text -text/plain -txt -Plain Text -Epic HTML'],
 ['71e9e1be-b0a1-47f1-aa0e-ddef34772ece', ' -txt'],
 ['c0783967-16b3-43a9-815a-38bdb13f63b7', ' -txt'],
 ['1e67ba8c-2a5f-4aeb-ae02-c9741b2b4f7a', 'application/rtf -rtf'],
 ['340785b4-6bc8-47b9-822e-3b144d34bf37', 'application/rtf -rtf'],
 ['5b2018e5-5fb9-46a9-bfa3-f13542d533bb', ' -txt'],
 ['ee36e911-eb03-4225-b47a-37703520174a', ' -Rich Text -Plain Text -txt'],
 ['58b6c44e-582f-46a3-977e-67516deed9fb', ' -txt'],
 ['b0b

In [14]:
df2=pd.DataFrame(results, columns=["patient_id", "FileNames"])
df2.to_csv("/Users/hussein.alolimat/Downloads/res.csv")

In [19]:
for i in results:
    if "Rich Text" in i[-1] or i[-1].strip()=="-txt":
        print("\t".join([i[0], i[-1].replace(" ","/"), "EMR"]))
    else:
        print("\t".join([i[0], i[-1].replace(" ","/"), "DOC"]))

3aba39e9-934d-44b9-92d2-4b1d67749272	application/rtf/-rtf	DOC
7ecb01df-4cdb-4f3a-a6a1-a6403d04ba31	/-Rich/Text/-txt	EMR
11520710-9145-4f67-8a79-e7ebf0c2ebba	/-txt	EMR
d48ff340-65e7-4de7-b9f2-439a20f11502	/-rtf	DOC
d88f8274-e9a2-4a6f-a190-27df6982a88a	/-pdf/-application/pdf/-txt	DOC
fc33de31-e8d9-46d6-b987-b49986109af7	/-txt	EMR
ceeac743-ed06-492c-a43b-87dad97be0e1	/-txt	EMR
a3cec039-87aa-4435-a6ea-07128bd6e5ba	application/rtf/-rtf	DOC
7432ca13-8e28-4a0b-a159-640213b4b327	Rich/Text/-text/plain/-txt/-Plain/Text/-Epic/HTML	EMR
71e9e1be-b0a1-47f1-aa0e-ddef34772ece	/-txt	EMR
c0783967-16b3-43a9-815a-38bdb13f63b7	/-txt	EMR
1e67ba8c-2a5f-4aeb-ae02-c9741b2b4f7a	application/rtf/-rtf	DOC
340785b4-6bc8-47b9-822e-3b144d34bf37	application/rtf/-rtf	DOC
5b2018e5-5fb9-46a9-bfa3-f13542d533bb	/-txt	EMR
ee36e911-eb03-4225-b47a-37703520174a	/-Rich/Text/-Plain/Text/-txt	EMR
58b6c44e-582f-46a3-977e-67516deed9fb	/-txt	EMR
b0b6ba20-e209-433f-8d9d-15bb1f979a91	application/rtf/-rtf	DOC
f2f16904-8f26-4918-bbcb-0d

In [41]:
for i, row in df.iterrows():
    qry = f"""
    SELECT partner_filename, mime_type
    FROM vault.clinical_document_gateway_meta_sat AS a
           JOIN vault.patient_clinical_document_link AS b
             ON a.clinical_document_hkey = b.clinical_document_hkey
        where b.patient_tempus_id='{row.patient_id}';
    """
    fileNames=set()
    for _, r in sqlio.read_sql_query(qry, conn_clq).iterrows():
        if len(r.partner_filename.strip()):
            fileNames.add(r.partner_filename.split(".")[-1])
        if len(r.mime_type.strip()):
            fileNames.add(r.mime_type)

    if "Rich Text" in fileNames or len(fileNames)==1 and next(iter(fileNames))=="txt":
        print("EMR")
    else:
        print("DOC")

{'application/rtf', 'rtf'}
DOC
{'Rich Text', 'txt'}
EMR
{'txt'}
EMR
{'rtf'}
DOC
{'pdf', 'application/pdf', 'txt'}
DOC
{'txt'}
EMR
